In [ ]:
from os import listdir

for filename in listdir('output'):
    if '_grid_search' in filename:
        print(filename)

In [ ]:
import pandas as pd

gs_res = pd.read_csv('output/lgb_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_res.head()

In [ ]:
import pandas as pd

gs_nn_res = pd.read_csv('output/nn_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_nn_res.head()

In [ ]:
stacknet_res = pd.read_csv('output/oof/stacknet_report.csv').sort_values(by=['oof_cv_score'])
stacknet_res

In [ ]:
# 1. oof_cv_score vs. gs_val_auc
# for NN, oof_cv_score != gs_val_auc is normal, because gs_val_auc is get by train+val in grid search, while 
# oof_cv_score is got by nfolds cv
# for LGB, if gs_cv is true in config file, then oof_cv_score == gs_val_auc (almost). otherwise, based on the same
# reason as NN, oof_cv_score != gs_val_auc is normal 

# 2. test_score
# for NN: the one(not showing in log) is got by train+val and load_weights when doing prediction after grid search
# (when do_preds is true in config file), and test_score is got during oof where all trainig data are used. It's 
# normal that they are not equal
# for LGB, since model is retrain using all training data when doing prediction after grid search, the one is not in 
# log == test_score

In [ ]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

In [ ]:
from automl_libs import BaseLayerResultsRepo

In [ ]:
repo = BaseLayerResultsRepo(['label'], 'output/oof/', load_from_file=True)

In [ ]:
for m,s in repo.show_scores():
    print(s,m)

In [ ]:
gs_res.head(1).T.to_dict()

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
lsvc = CalibratedClassifierCV(LinearSVC())

In [ ]:
lsvc.fit(np.array([0,1,2,3,4,5]).reshape(-1,1), [1,0,0,1,1,0])

In [ ]:
lsvc.predict_proba(2.22)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
svc = SVC(kernel='linear', probability=True)#, class_weight='auto')


In [ ]:
svc.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

In [ ]:
svc.predict_proba(2.22)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
clf = OneVsRestClassifier(BaggingClassifier(svc, max_samples=1.0 / 5, n_estimators=5))
clf.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

In [ ]:
clf.predict_proba(2.22)